<a href="https://colab.research.google.com/github/Dylanfm204/Optimizacion-Hiperparametros-DylanFlores/blob/main/Optimizacion_Hiperparametros_DylanFlores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/10-Optimizacion-Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Escuela de Ciencias Físicas y Matemática</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Optmización de Hiperparámetros
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Andrés Merino &bull; 2024-02
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

A lo largo de este taller, aplicaremos optimización de hiperparámetros en un modelo que elijas.

Los paquetes necesarios son:

In [1]:
# Paquetes necesarios
import pandas as pd  # Manejo de datos
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import joblib

---
## <font color='264CC7'> Clasificación </font>


### <font color='264CC7'> Preprocesamiento de datos </font>

Primero necesitas el conjunto de datos. Los datos a utilzar son los seleccionados en la clase anterior.

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Carga el conjunto de datos y procésalos:
<ul>
  <li>Muestra algunos datos.</li>
  <li>Muestra una descripción de los datos.</li>
  <li>Escala los datos si es necesario.</li>
</ul>
</div>

In [2]:
# Instalar gdown en caso de que no esté disponible
!pip install gdown --quiet

# Descargar el archivo desde el link público de Google Drive
import gdown

# Usar el ID del archivo proporcionado
file_id = "1fIoyvM7Ow0hZoq5ThBetJyOzSgxSDYfP"
url = f"https://drive.google.com/uc?id={file_id}"

# Nombre del archivo a guardar
output = "FakeBills-dataset.csv"

# Descargar el archivo
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1fIoyvM7Ow0hZoq5ThBetJyOzSgxSDYfP
To: /content/FakeBills-dataset.csv
100%|██████████| 65.5k/65.5k [00:00<00:00, 5.73MB/s]


'FakeBills-dataset.csv'

In [3]:
# Cargar el dataset descargado
df = pd.read_csv(output)
# Cargar el dataset descargado con el separador adecuado
df = pd.read_csv(output, sep=';')
# Mostrar los primeros registros
print("Primeros registros del dataset:")
print(df.head())

# Mostrar una descripción de los datos
print("\nDescripción estadística del dataset:")
print(df.describe())

Primeros registros del dataset:
   is_genuine  diagonal  height_left  height_right  margin_low  margin_up  \
0        True    171.81       104.86        104.95        4.52       2.89   
1        True    171.46       103.36        103.66        3.77       2.99   
2        True    172.69       104.48        103.50        4.40       2.94   
3        True    171.36       103.91        103.94        3.62       3.01   
4        True    171.73       104.28        103.46        4.04       3.48   

   length  
0  112.83  
1  113.09  
2  113.16  
3  113.51  
4  112.54  

Descripción estadística del dataset:
          diagonal  height_left  height_right   margin_low    margin_up  \
count  1500.000000  1500.000000   1500.000000  1463.000000  1500.000000   
mean    171.958440   104.029533    103.920307     4.485967     3.151473   
std       0.305195     0.299462      0.325627     0.663813     0.231813   
min     171.040000   103.140000    102.820000     2.980000     2.270000   
25%     171.750000  

In [4]:
# Verificar si hay valores faltantes
print("\nValores faltantes por columna:")
print(df.isnull().sum())


Valores faltantes por columna:
is_genuine       0
diagonal         0
height_left      0
height_right     0
margin_low      37
margin_up        0
length           0
dtype: int64


In [5]:
# Manejar valores faltantes (rellenarlos con la mediana)
df.fillna(df.median(), inplace=True)

In [6]:
# Verificar nuevamente valores faltantes después de la imputación
print("\nValores faltantes después de la imputación:")
print(df.isnull().sum())


Valores faltantes después de la imputación:
is_genuine      0
diagonal        0
height_left     0
height_right    0
margin_low      0
margin_up       0
length          0
dtype: int64


In [7]:
# Definir X (variables predictoras) y y (variable objetivo)
X = df.drop(columns=['is_genuine'])  # Todas las columnas excepto la variable objetivo
y = df['is_genuine']  # Variable objetivo (True o False)

In [8]:
# Dividir el dataset en entrenamiento y prueba con estratificación porque y es categórica
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Verificar la distribución en los conjuntos
print("Distribución en y_train:\n", y_train.value_counts())
print("\nDistribución en y_test:\n", y_test.value_counts())

Distribución en y_train:
 is_genuine
True     800
False    400
Name: count, dtype: int64

Distribución en y_test:
 is_genuine
True     200
False    100
Name: count, dtype: int64



### <font color='264CC7'> Modelo </font>


<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Selecciona el mejor modelo de las clases anteriores.
<ul>
  <li>Muestra los hiperparámetros del modelo.</li>
  <li>Consulta qué significan al menos 4 hiperparámetros.</li>
  <li>Selecciona los hiperparámetros que deseas optimizar, al menos 3.</li>
</ul>
</div>

In [9]:
# Crear y entrenar un arbol con ganancia de información
modelo_base = RandomForestClassifier(random_state=54)

# Parámetros del modelo
modelo_base.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 54,
 'verbose': 0,
 'warm_start': False}


Para optimizar el modelo RandomForestClassifier, seleccionaremos tres hiperparámetros clave. Primero, n_estimators, que determina la cantidad de árboles en el bosque; un mayor número de árboles puede mejorar la estabilidad del modelo, pero aumenta el costo computacional, por lo que probaremos con valores de [10, 50, 100]. Luego, max_depth, que define la profundidad máxima de los árboles y ayuda a controlar el sobreajuste; exploraremos los valores [None, 5, 10, 20] para evaluar el impacto en la generalización del modelo. Finalmente, min_samples_split, que establece el número mínimo de muestras necesarias para dividir un nodo y afecta la complejidad del modelo, probando valores de [2, 5, 10] para encontrar el equilibrio entre sobreajuste y subajuste.

### <font color='264CC7'> Optimización por GridSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica GridSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 3 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [10]:
# Definir los hiperparámetros a optimizar
parametros = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# Configurar validación cruzada con 5 folds
k_fold = KFold(n_splits=5, shuffle=True, random_state=54)

# Aplicar GridSearchCV
grid_search = GridSearchCV(modelo_base, parametros, cv=k_fold, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=54, shuffle=True),
             estimator=RandomForestClassifier(random_state=54), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 50, 100]},
             scoring='accuracy')

In [11]:
# Mostrar los mejores hiperparámetros y la mejor puntuación obtenida
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor score:", grid_search.best_score_)

Mejores hiperparámetros: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 10}
Mejor score: 0.9925


En los resultados de la optimización de hiperparámetros, el modelo encontró que la mejor combinación de valores para RandomForestClassifier fue max_depth=None, min_samples_split=2 y n_estimators=10, obteniendo una precisión del 99.25% en la validación cruzada. Esto indica que el modelo generaliza bien con una estructura de árboles sin una profundidad máxima predefinida, permitiendo que cada árbol crezca hasta alcanzar la mejor separación posible, con un mínimo de dos muestras necesarias para dividir un nodo, lo que mantiene suficiente flexibilidad en la partición de datos.

In [12]:
from sklearn.metrics import accuracy_score, classification_report

# Usar el mejor modelo encontrado por GridSearchCV
modelo_optimo = grid_search.best_estimator_

# Hacer predicciones en el conjunto de prueba
y_pred = modelo_optimo.predict(X_test)

# Evaluar el desempeño
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy en el conjunto de prueba:", accuracy)

# Mostrar reporte de clasificación
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred))

Accuracy en el conjunto de prueba: 0.9866666666666667

Reporte de clasificación:
               precision    recall  f1-score   support

       False       0.99      0.97      0.98       100
        True       0.99      0.99      0.99       200

    accuracy                           0.99       300
   macro avg       0.99      0.98      0.98       300
weighted avg       0.99      0.99      0.99       300



Al evaluar el modelo optimizado en el conjunto de prueba, se obtuvo una precisión global del 98.67%, lo que confirma un alto rendimiento. En términos de clasificación, la clase "False" (billetes falsos) presenta un recall del 97%, lo que indica que el modelo identificó correctamente la mayoría de los casos, aunque con un leve margen de error. En cambio, la clase "True" (billetes genuinos) tuvo un recall perfecto del 99%, asegurando que casi todos los billetes reales fueron clasificados correctamente. En general, el F1-score de ambas clases es alto (0.99), lo que demuestra un excelente equilibrio entre precisión y recall. Esto sugiere que el modelo es confiable para la detección de billetes falsos, minimizando tanto los falsos positivos como los falsos negativos.

### <font color='264CC7'> Optimización por RandomSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica RandomSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 5 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Usa RandomSearchCV con 25 iteraciones.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [13]:
# Definir los hiperparámetros a optimizar con al menos 5 valores por parámetro
parametros_random = {
    'n_estimators': [10, 50, 100, 150, 200],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': ['sqrt', 'log2', None]
    }

In [14]:
# Configurar validación cruzada con 5 folds
k_fold = KFold(n_splits=5, shuffle=True, random_state=54)

# Aplicar RandomizedSearchCV con 25 iteraciones
random_search = RandomizedSearchCV(
    modelo_base,
    parametros_random,
    n_iter=25,
    cv=k_fold,
    scoring='accuracy',
    random_state=54,
    n_jobs=-1
)

In [15]:
# Ajustar el modelo con los datos de entrenamiento
random_search.fit(X_train, y_train)

# Obtener los mejores parámetros y la mejor puntuación
mejores_parametros_random = random_search.best_params_
mejor_score_random = random_search.best_score_
print("Mejores hiperparámetros:", mejores_parametros_random)
print("Mejor score:", mejor_score_random)

Mejores hiperparámetros: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 20}
Mejor score: 0.9916666666666666


En cuanto a los resultados obtenidos, el mejor conjunto de hiperparámetros encontrado fue n_estimators=100, min_samples_split=10, min_samples_leaf=2, max_features='log2', y max_depth=20, logrando un score de 0.9916 en validación cruzada. Esto indica que un bosque con 100 árboles, profundidad limitada a 20 niveles y división de nodos con al menos 10 muestras proporciona el mejor balance entre rendimiento y generalización. La configuración de max_features='log2' sugiere que reducir la cantidad de atributos evaluados en cada división mejora la estabilidad del modelo, evitando ruido y sobreajuste. Estos resultados demuestran que RandomizedSearch puede encontrar hiperparámetros eficientes en menos iteraciones en comparación con GridSearch, permitiendo una exploración más amplia del espacio de búsqueda sin un costo computacional excesivo.

In [16]:
# Definir el modelo con los mejores hiperparámetros
modelo_optimo = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=2,
    max_features='log2',
    random_state=54
)

# Entrenar el modelo en el conjunto de entrenamiento
modelo_optimo.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = modelo_optimo.predict(X_test)

# Evaluar el rendimiento del modelo
precision = accuracy_score(y_test, y_pred)
print(f"Precisión en el conjunto de prueba: {precision:.4f}")

# Mostrar el reporte de clasificación
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred))

Precisión en el conjunto de prueba: 0.9867

Reporte de clasificación:
               precision    recall  f1-score   support

       False       0.99      0.97      0.98       100
        True       0.99      0.99      0.99       200

    accuracy                           0.99       300
   macro avg       0.99      0.98      0.98       300
weighted avg       0.99      0.99      0.99       300




El modelo optimizado con los mejores hiperparámetros fue entrenado y evaluado en el conjunto de prueba, obteniendo una alta precisión. Con n_estimators=100, max_depth=20, min_samples_split=10, min_samples_leaf=2 y max_features='log2', el RandomForestClassifier mostró un buen desempeño en la clasificación, asegurando que los árboles no sean ni demasiado profundos ni demasiado restrictivos en la división de nodos. La precisión obtenida en el conjunto de prueba refleja la capacidad del modelo para generalizar correctamente, minimizando errores de clasificación. El reporte de clasificación proporcionó métricas detalladas como precision, recall y F1-score, lo que permite verificar que el modelo es confiable para la detección de billetes falsos. El alto desempeño del modelo optimizado demuestra que RandomizedSearchCV permitió encontrar combinaciones de hiperparámetros más eficientes en comparación con un enfoque manual, logrando un balance entre complejidad y precisión.

### <font color='264CC7'> Guardado de modelo </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Con los parámetros óptimos que mejor resultado dieron, reentrena el modelo, muestra su score y guárdalo.
</div>

In [17]:
# Definir el modelo con los mejores hiperparámetros
modelo_final = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=2,
    max_features='log2',
    random_state=54
)

# Reentrenar el modelo con todos los datos de entrenamiento
modelo_final.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
y_pred_final = modelo_final.predict(X_test)
precision_final = accuracy_score(y_test, y_pred_final)

# Guardar el modelo entrenado
modelo_path = "modelo_random_forest.pkl"
joblib.dump(modelo_final, modelo_path)

# Mostrar la precisión del modelo final
precision_final, modelo_path

(0.9866666666666667, 'modelo_random_forest.pkl')

In [18]:
# Reentrenar el modelo con los mejores hiperparámetros encontrados por RandomizedSearchCV
modelo_final_random = RandomForestClassifier(
    **mejores_parametros_random,  # Usar los hiperparámetros óptimos encontrados
    random_state=54
)

# Ajustar el modelo en el conjunto de entrenamiento
modelo_final_random.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
y_pred_final_random = modelo_final_random.predict(X_test)
precision_final_random = accuracy_score(y_test, y_pred_final_random)

# Guardar el modelo entrenado con los hiperparámetros optimizados por RandomizedSearchCV
modelo_random_path = "modelo_random_forest_randomized.pkl"
joblib.dump(modelo_final_random, modelo_random_path)

# Mostrar la precisión final y la ruta donde se guardó el modelo
precision_final_random, modelo_random_path

(0.9866666666666667, 'modelo_random_forest_randomized.pkl')

### <font color='264CC7'> Publicación </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Coloca el este cuaderno y el modelo en tu repositorio de GitHub. Agrega una licencia MIT y un README.md donde se explique el contenido del repositorio, los datos utilizados y los resultados obtenidos.
</div>

Link : https://github.com/Dylanfm204/Optimizacion-Hiperparametros-DylanFlores